In [10]:
import anvil.server
import anvil.media
from anvil.tables import app_tables
from anvil import BlobMedia
anvil.server.connect("4LWTNNDYE2AJJJCWPGPFAFHX-CLFFCZ4AB5DTBE3B")

In [11]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (20,10)

In [12]:
import re
import cv2 
from pathlib import Path
import pytesseract as pytesseract
from PIL import Image
import sys
from pdf2image import convert_from_path
import os
from io import StringIO
import imutils
import os
import pickle
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import ShuffleSplit
from sklearn.feature_extraction.text import CountVectorizer

In [13]:
def pdf_img(file, name):
    pages = convert_from_path(file, 600)
    image_counter=0
    for page in pages:
        image_counter += 1
        filename = name[:-4]+"_"+str(image_counter)+".jpg"

        # Save the image of the page in system
        page.save(filename, 'JPEG')

        # Increment the counter to update filename
        
        
    return image_counter

def crop(img, pge):
    
    kernel1 = np.ones((3,5),np.uint8)
    kernel2 = np.ones((9,9),np.uint8)

    imgGray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    imgBW=cv2.threshold(imgGray, 230, 255, cv2.THRESH_BINARY_INV)[1]

    img1=cv2.erode(imgBW, kernel1, iterations=1)
    img2=cv2.dilate(img1, kernel2, iterations=3)

    #plt.imshow(img2)
    #plt.show()

    imgLines= cv2.HoughLinesP(img2,15,np.pi/180,10, minLineLength = 3000, maxLineGap = 100)

    print(imgLines)
    for i in range(len(imgLines)):
        for x1,y1,x2,y2 in imgLines[i]:
            cv2.line(img,(x1,y1),(x2,y2),(0,255,0),2)
        
    #plt.imshow(img)
    #plt.show()
    h, w, c = img.shape
    roi = img[imgLines[0][0][1]:, :-600]
    cv2.imwrite('roi'+ str(pge)+'.jpg',roi)
    
    return

# noise removal
def preprocess(image):
    image = cv2.medianBlur(image,5)

    kernel = np.ones((5,5),np.uint8)
    image = cv2.dilate(image, kernel, iterations = 1)
    
    kernel = np.ones((5,5),np.uint8)
    return cv2.erode(image, kernel, iterations = 1)


def output_file(image_lst, name):
    # Creating a text file to write the output
    outfile = name[:-4] + ".txt"
    
    # Open the file in append mode so that 
    # All contents of all images are added to the same file
    f = open(outfile, "w")
    custom_config = r'--oem 1 --psm 6' #can play around with this
    text = ''
    for i in image_lst:
        text = text + str(((pytesseract.image_to_string(i, config=custom_config))))

    f.write(text)

    # Close the file after writing all the text.
    f.close()
    
    return text

def get_df(outfile):
    pn = []
    hn = []
    name = []
    qty = []
    count_pn = 0
    count_hn = 0
    with open (outfile, 'rt') as myfile:  # Open lorem.txt for reading text
        for myline in myfile:              # For each line, read to a string,
            myline = myline.strip('\n')
            lst = myline.split()
            
            if count_hn > count_pn: #check for subcomponent with multiple heat numbers
                last = pn[-1]
                pn.append(last)
                last = name[-1]
                name.append(last)
                count_pn += 1
                
            if len(lst) == 0:
                continue
            elif re.search("W-G", myline):
            #elif myline[0] == 'W' and myline[1] == '-':
                hn.append(lst[1])
                qty.append(lst[2])
                count_hn += 1
                continue
                
            a = re.search(".{4,5}\-([0-9]){3,4}\-([0-9]){4,5}", myline)
            b = re.search("[0-9]{9}", myline)
            c = re.search("[A-Z][0-9]{5,9}", myline)
            if bool(a) or bool(b) or bool(c):
                full = ' '.join(lst[1:-2])
                name.append(full)
                if bool(a):
                    pn.append(a.group(0))
                elif bool(b):
                    pn.append(b.group(0))
                else:
                    pn.append(c.group(0))
                count_pn += 1
            
    df = pd.DataFrame(list(zip(name, pn, hn, qty)),
               columns =['Name','Part Number', 'Heat Number', 'Quantity'])
    return df

In [14]:
@anvil.server.callable
def extract(file, name):
    #anvil.media.download(wo)
    with anvil.media.TempFile(file) as filepath:
        pages = pdf_img(filepath, name)
    print("Gotten pages!")
    img = []
    for i in range(pages): 
        img.append(cv2.imread(name[:-4]+'_' + str(i+1) +'.jpg'))
    print('Converted and saved WO as image!')
    
    for i in range(len(img)):
        crop(img[i], i+1)
    img = []
    print('Cropped images!')
    
    for i in range(pages):
        pic = cv2.imread('roi' + str(i+1) +'.jpg')
        img.append(preprocess(pic))
    print('Extracted text! Putting in dataframe!')
    
    out = output_file(img, name)
    df = get_df(name[:-4]+".txt")
    print("Dataframe created!")
    
    for i in range(len(df)):
        app_tables.wo_components.add_row(Name = df['Name'][i], PN = df['Part Number'][i], HN = df['Heat Number'][i],
                          Qty = df['Quantity'][i])
    
    return True
    

@anvil.server.callable
def output_excel(name):
    datafile = app_tables.wo_components.search().to_csv()
    
    with anvil.media.TempFile(datafile) as filename:
        df = pd.read_csv(filename)
        df.drop('ID', axis = 1, inplace = True)
        print(df)
    print("Read in datatable!")
    
    base = str(os.path.join(Path.home()))
    
    fp = newpath2
    
    #path_to_desktop = str(os.path.join(Path.home(), "Desktop"))
    df.to_csv(fp + name[:-4] + '.csv', index = False)
    print('Excel created. Success!')
    
    return True


In [15]:
#categories

classes = ['Drift report', 'NDE report', 'Threading Cert', 'Work order', 'Traceability log sheet',
          'Hardness report', 'Material Certs', 'Certificate of conformance', 'Diagrams',
          'Pressure test', 'Delivery invoice']

category_codes = {
    'Drift report': 0,
    'Hardness report': 1,
    'Material Certs': 2,
    'NDE report': 3,
    'Threading Cert': 4,
    'Work order': 5,
    'Traceability log sheet': 6,
    'Pressure test': 7,
    'Diagrams': 8,
    'Certificate of conformance': 9,
    'Delivery invoice':10
}

#base = r'C:\Users\JCheng10\OneDrive - Schlumberger\Desktop\Classification'
base = str(os.path.join(Path.home()))
    
newpath = base + '//classification_results//'
if not os.path.exists(newpath):
    os.makedirs(newpath)
    
newpath2 = base + '//extraction_results//'
if not os.path.exists(newpath2):
    os.makedirs(newpath2)

# load the model from disk
filename = 'finalized_model.sav'
model = pickle.load(open(filename, 'rb'))
file = 'vectorizer.sav'
vectorizer = pickle.load(open(file, 'rb'))
file2 = 'vectorizer_gib.sav'
vectorizer_gib = pickle.load(open(file2, 'rb'))
file3 = 'gibberish.sav'
gibberish = pickle.load(open(file3, 'rb'))

In [16]:
category_names = {
    0:'Drift report',
    1:'Hardness report',
    2:'Material Certs',
    3:'NDE report',
    4:'Threading Cert',
    5:'Work order',
    6:'Traceability log sheet',
    7:'Pressure test',
    8:'Diagrams',
    9:'Certificate of conformance',
    10:'Delivery invoice'
}

In [19]:
def create_csv(path):

    files = os.listdir(path)
    
    img_list = []
    
    for filename in files:
        if filename.lower().endswith(('.jpg', '.jpeg')):
            img_list.append(filename)
    
    df = pd.DataFrame()
   
    df['images']=img_list
        
    return df

def convert_to_image(path, name):
    pages = convert_from_path(path, 500)
    image_counter=1
    for page in pages:
        filename = name[:-4] + '_' + str(image_counter)+".jpg"

        # Save the image of the page in system
        page.save(newpath + filename, 'JPEG')

        # Increment the counter to update filename
        image_counter += 1
    return pages

def preprocess(image):
    image = cv2.medianBlur(image,5)

    kernel = np.ones((5,5),np.uint8)
    image = cv2.dilate(image, kernel, iterations = 1)
    
    kernel = np.ones((5,5),np.uint8)
    return cv2.erode(image, kernel, iterations = 1)

def extract_text(df):
    df['text'] = ''
    for i in range(len(df)):
        # Creating a text file to write the output
        #outfile = df['document'][i][:-4] + ".txt"

        # Open the file in append mode so that 
        # All contents of all images are added to the same file
        #f = open(outfile, "w")
        #custom_config = r'--oem 3 --psm 6' #read normally line by line
        img = cv2.imread(newpath + df['images'][i])
        img = preprocess(img)
        custom_config = r'-c tessedit_char_blacklist=0123456789 --psm 6' #dont read numbers
        text = str(((pytesseract.image_to_string(img, config=custom_config))))

        #f.write(text)

        # Close the file after writing all the text.
        #f.close()
        textf = check_text(text, img)
        df['text'][i] = textf

def data_cleaning(data):
    #cleaning
    #\r and \n
    data['text_1'] = data['text'].str.replace("\r", " ")
    data['text_1'] = data['text_1'].str.replace("\n", " ")
    data['text_1'] = data['text_1'].str.replace("    ", " ")

    # " when quoting text
    data['text_1'] = data['text_1'].str.replace('"', '')

    # Lowercasing the text
    data['text_1'] = data['text_1'].str.lower()

    punctuation_signs = list("?:!.,;")

    for punct_sign in punctuation_signs:
        data['text_1'] = data['text_1'].str.replace(punct_sign, '')

    data['text_1'] = data['text_1'].str.replace("'s", "")

def put_into_folders(df):
    df['Category_Predicted'] = df['predictions']
    df = df.replace({'Category_Predicted':category_names})
    for i in range(len(df)):
        Path(newpath+df['images'][i]).rename(newpath+df['Category_Predicted'][i]+'//'+df['images'][i])

def check_text(text, img):
    custom_config = r'-c tessedit_char_blacklist=0123456789 --psm 6'
    text_tfm = [text]
    if gibberish.predict(vectorizer_gib.transform(text_tfm)) == 1:
        return text
    else:
        img1 = cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE)
        text1 = str(((pytesseract.image_to_string(img1, config=custom_config))))
        text_tfm = [text1]
        if gibberish.predict(vectorizer_gib.transform(text_tfm)) == 1:
            return text1
        else: 
            img2 = cv2.rotate(img, cv2.ROTATE_90_COUNTERCLOCKWISE)
            text2 = str(((pytesseract.image_to_string(img2, config=custom_config))))
            return text2

def convert_combine(name):
    for i in os.listdir(newpath):
        indiv = newpath + i
        if len(os.listdir(indiv)) > 1:
            imglist = []
            #numlist = []
            for j in os.listdir(indiv):
                if j.lower().endswith(('.jpg', '.jpeg')):
                    im = Image.open(indiv+'//'+j)
                    im.convert('RGB')
                    imglist.append(im)
                
            im1 = imglist.pop(0)
            newname = indiv+'//'+name+'.pdf'
            im1.save(newname, "PDF" ,resolution=500, save_all=True, append_images=imglist)
        elif len(os.listdir(indiv)) == 1:
            if os.listdir(indiv)[0].lower().endswith(('.jpg', '.jpeg')):
                im = Image.open(indiv+'//'+ os.listdir(indiv)[0])
                im.convert('RGB')
                newname = indiv+'//'+name #+'.pdf'
                im.save(newname, "PDF" ,resolution=500)
            
        for file in os.listdir(indiv):
            if file.endswith(('.jpg', '.jpeg')):
                os.remove(indiv+'//' + file)

In [20]:
@anvil.server.callable

def classify(file, filename): 
    with anvil.media.TempFile(file) as filepath:
        pages = convert_to_image(filepath, filename) 
    print("Images formed!")
    
    df = pd.DataFrame()
    df = create_csv(newpath)
    print("Dataframe created!")
    
    
    extract_text(df)
    print("Text extracted!")
    
    #return df
    
    
    data_cleaning(df)
    print("Text cleaned!")
    
    
    model_input = vectorizer.transform(df.text_1)
    
    preds = model.predict(model_input)
    print("Model succeeded!")
    
    
    df['predictions'] = preds
    print("Dataframe updated!")
    
    #return df
    
    #making individual folders after splitting and categorising
  
    for i in classes:
        folders = newpath + i
        if not os.path.exists(folders):
            os.makedirs(folders)

    put_into_folders(df)
    print("All files split into their respective folders!")
    
    convert_combine(filename)
    print("Converted and combined too!")

    return True

#anvil.server.wait_forever()